In [60]:
import numpy as np
from PIL import Image,ImageOps
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:

def convertImageToArrayRGB(imagefile, grayscale):
    """
    Input: Image .JPG file
    Output: 2D array representing the image.
    """
    imageObject = Image.open(imagefile)
    if not grayscale :
        return np.array(imageObject)
    else :
        gray_image = ImageOps.grayscale(imageObject)
        return np.array(gray_image)
    

img_2 = convertImageToArrayRGB('L4.jpg', False)
img_2_Gray = convertImageToArrayRGB('L4.jpg', True)
img_2_Gray


array([[27, 27, 27, ..., 27, 27, 27],
       [27, 28, 27, ..., 27, 27, 27],
       [27, 28, 27, ..., 27, 27, 27],
       ...,
       [32, 35, 35, ..., 40, 40, 39],
       [32, 35, 35, ..., 38, 39, 40],
       [32, 34, 35, ..., 38, 38, 39]], dtype=uint8)

In [62]:
def CalculateIntegral(arrayImage):
    """
    Input: 2D array representing the image (feel free to use a predefined function to transform an image into an array).
    Output: 2D array representing the integral image.
    """
    height = len(arrayImage)
    width = len(arrayImage[0])
    
    #Init a 3d array of length, width and RGB values
    si = np.zeros(arrayImage.shape)
    si.astype(int)
    si.setflags(write=1)

    #Add First Coloumn to SI
    for i in range(0,height):
        si[i][0] =  arrayImage[i][0]

    #Get SI    
    for i in range(0,height):
        for j in range(1,width):
            si[i][j] = arrayImage[i][j] + si[i][j-1]
    #Get II
    for j in range(0,width):
        for i in range(1,height):
            si[i][j] = si[i][j] + si[i-1][j]       
    
    return si

In [63]:
#si = CalculateIntegral(img_2)
si_g = CalculateIntegral(img_2_Gray)
print(si_g[2159][3839])

544235163.0


In [70]:
def CalculateLocalSum(intergralImage, left_corner, right_corner):
    """
    Input: An integral image, and a two pairs of coordinates (𝑝0=(𝑥0,𝑦0),𝑝1=(𝑥1,𝑦1)).
    Output: The local sum for the rectangular area defined by the pair of points (as 𝑝0 being the upper left corner, and 𝑝1 being the lower right corner of this                  rectangular area).
    """
    # Returns Intensity Relative to Origin
    if(left_corner==0 and right_corner!=-1):
        if right_corner[0] == 0 or right_corner[1] == 0:
            #print("BEDANI",intergralImage[right_corner[1]][right_corner[0]],intergralImage[0][0])
            return intergralImage[right_corner[1]][right_corner[0]] - intergralImage[0][0]
        return intergralImage[right_corner[1]][right_corner[0]] - intergralImage[1][right_corner[0]] - intergralImage[right_corner[1]][1] + intergralImage[1][1]
    #Returns Difference between right and left indicies.
    else:
        #hnafge zero to one
        #print(left_corner,right_corner)
        return intergralImage[right_corner[1]][right_corner[0]] + intergralImage[left_corner[1]][left_corner[0]] - intergralImage[left_corner[1]][right_corner[0]]  - intergralImage[right_corner[1]][left_corner[0]] 
        


#print(CalculateLocalSum(si,(300,300),(500,500)))
test1 = CalculateLocalSum(si_g,(300,300),(500,500))
test2 = si_g[500][500]+si_g[300][300]-si_g[300][500]-si_g[500][300]
print(test1==test2)
print(CalculateLocalSum(si_g,0,(500,500)))

True
6854969.0


In [65]:
def kernelGenerator(size,kernelType):
    """
    Input: The size of the Kernel and Its Type
    Output: returns the requested kernel and describes it breifly
    """
    if(size%2 ==0 or size ==1):
        print("Size has to be an Odd Number >1")
        return

    middle_row = math.floor(size/2)    
    if(kernelType.lower()=="prewitt"):
        print("Returning Prewitt of Size: %s" %size)

        #Computin Value of H1
        h1 = np.zeros(size**2).reshape(size,size)
        h1[0:middle_row] = [1]*size
        h1[middle_row] = [0]*size
        h1[middle_row+1:] = [-1]*size

        #Computing value of H2
        h2 = np.copy(h1)
        h2 = np.transpose(h2)

        return h1,h2

    elif(kernelType.lower()=="laplace"):
        print("Returning Laplace of Size: %s "%size)
        h1 = np.zeros(size**2).reshape(size,size) - 1
        h1[middle_row][middle_row] = (size**2) -1
        return h1,[]

    elif(kernelType.lower()=="mean"):
        print("Returning Mean of Size: %d "%size)
        h1 = (np.zeros(size**2).reshape(size,size) + 1) * (1/(size**2))
        return h1,[]
    else:
        print("This Kernel Is Not Supported")
        return [],[]

kernelGenerator(5,"prewitt")

Returning Prewitt of Size: 5


(array([[ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.]]),
 array([[ 1.,  1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1., -1.]]))

In [73]:
#print(CalculateLocalSum(si_g,0,(500,500)))
#array[height][width]

def EdgeDetect(integral_image, kernel_size):
    if(kernel_size%2 ==0 or kernel_size ==1):
        print("Size has to be an Odd Number >1")
        return

    print("Integral Image: \n",integral_image)
    width = len(integral_image[0])
    height = len(integral_image)
    print(width,height)



    image =  Image.fromarray(integral_image)
    #image.show()
    localSumMatrix = np.copy(integral_image)
    stepSize = math.floor(kernel_size/2)
    h1_result = np.copy(localSumMatrix) 
    h2_result = np.copy(localSumMatrix) 
    magn = np.copy(localSumMatrix) 
    laplace_result = np.copy(localSumMatrix)

    print("StepSize",stepSize)
   # for j in range(0,height):
   #     for i in range(0,width):
   #         localSumMatrix[j][i]= CalculateLocalSum(integral_image,0,(i,j))
    
    
    for j in range(kernel_size,height-kernel_size):
        for i in range(kernel_size,width-kernel_size):
                   #print("pixel @ ",i,j)
                   X1 = int(i - (stepSize/2))
                   Y1 = int(j - (stepSize/2))
                   Y2 = int(j - (stepSize/2))
                   X2 = int(i + (stepSize/2))
                   h1_upper = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))

                   Y1 = int(j + (stepSize/2))
                   Y2 = int(j + (stepSize/2))
                   h1_lower = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))
                   h1_all = (h1_upper +(h1_lower*-1)) #check if we need to divide
                   h1_result[j][i] = int(h1_all)

                   X1 = int(i - (stepSize/2))
                   Y1 = int(j - (stepSize/2))
                   Y2 = int(j + (stepSize/2))
                   X2 = int(i - (stepSize/2))
                   h2_left = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))

                   X1 = int(i + (stepSize/2))
                   X2 = int(i + (stepSize/2))

                   h2_right = CalculateLocalSum(integral_image,(X1,Y1),(X2,Y2))
                   h2_all = (h2_right +(h2_left*-1)) #check if we need to divide
                   h2_result[j][i] = int(h2_all)
                   
                   magn[j][i] = math.sqrt((h2_result[j][i])**2 + (h1_result[j][i])**2)
    

    print("Local Sum Matrix: \n",localSumMatrix)
    print("Magnitude: \n",magn)  
    print(localSumMatrix==integral_image)
    image = Image.fromarray(magn)
    image.show()
    
    return #(h1,h2),(h1_abs,h2_abs)

test7 = CalculateIntegral(np.arange(25).reshape(5,5))

#EdgeDetect(test,5)
EdgeDetect(si_g,5)     

Integral Image: 
 [[2.70000000e+01 5.40000000e+01 8.10000000e+01 ... 1.04737000e+05
  1.04764000e+05 1.04791000e+05]
 [5.40000000e+01 1.09000000e+02 1.63000000e+02 ... 2.10170000e+05
  2.10224000e+05 2.10278000e+05]
 [8.10000000e+01 1.64000000e+02 2.45000000e+02 ... 3.15913000e+05
  3.15994000e+05 3.16075000e+05]
 ...
 [5.31360000e+04 1.40528000e+05 2.28945000e+05 ... 5.43803854e+08
  5.43887226e+08 5.43938425e+08]
 [5.31680000e+04 1.40595000e+05 2.29047000e+05 ... 5.43952348e+08
  5.44035759e+08 5.44086998e+08]
 [5.32000000e+04 1.40661000e+05 2.29148000e+05 ... 5.44100436e+08
  5.44183885e+08 5.44235163e+08]]
3840 2160
StepSize 2
Local Sum Matrix: 
 [[0.00000000e+00 2.70000000e+01 5.40000000e+01 ... 1.04710000e+05
  1.04737000e+05 1.04764000e+05]
 [2.70000000e+01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.40000000e+01 0.00000000e+00 2.70000000e+01 ... 1.05688000e+05
  1.05715000e+05 1.05742000e+05]
 ...
 [5.31090000e+04 0.00000000e+00 8.83630